In [1]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import os
from IPython.display import display
import tensorflow as tf
import math
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.utils import np_utils
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from collections import defaultdict
from keras.optimizers import Adam,SGD
from keras.layers import Dense, Embedding, LSTM, GRU
from keras import initializers
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.layers import GlobalAveragePooling2D
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from keras.models import Model
from keras.callbacks import Callback
from keras.layers import Conv2D, MaxPool2D
from keras import regularizers

Using TensorFlow backend.


In [2]:
NB_IV3_LAYERS_TO_FREEZE=150

In [3]:
load=np.load("setX0.npz")
temp=load['a']
temp=np.pad(temp,((0,0),(4,5),(4,5),(0,0)),mode='constant')
#temp=temp.astype(np.float16)
Xtrain=temp
ytrain=[]
ytrain.extend(load['b'])
for i in range(1,5):
    load=np.load("setX"+str(i)+".npz")
    temp=load['a']
    temp=np.pad(temp,((0,0),(4,5),(4,5),(0,0)),mode='constant')
    #temp=temp.astype(np.float16)
    #temp+=128
    #temp/=255.
    Xtrain=np.vstack((Xtrain,temp))
    ytrain.extend(load['b'])
print Xtrain.shape
ytrain=np.array(ytrain)
print ytrain.shape

(161000, 229, 229, 3)
(160000,)


In [4]:
Xtrain=Xtrain[:160000]

In [5]:
print Xtrain.shape

(160000, 229, 229, 3)


In [6]:
counte=defaultdict(int)
for i in ytrain:
    counte[i]+=1
items=counte.items()
items=np.array(items)
print items.shape
keep_labels=[]
for i,j in items:
    #print j
    if int(j)>1000:
        #print j
        keep_labels.append(i)
print keep_labels
print len(keep_labels)

(70, 2)
['Books & Stationery', 'Strength Training Equipment', 'Hair Care', 'Outdoor Activities', 'Bedding', 'Cameras', 'Storage', 'Components & Spare Parts', 'Bath & Body', 'Personal Care', 'Peripherals & Accessories', 'Large Appliances', 'Small Appliances', 'Baby & Toddler', 'Video Games', 'Western Clothing', 'Lingerie & Sleepwear', "Boys' Fashion", 'Lighting', 'Winter Clothing', 'Shoes & Clothing', "Babies' Fashion", 'Audio & Video', "Men's Shoes", 'Home Improvements', 'Furniture', 'Fitness Accessories', 'Skin Care', 'Shoes', 'Gaming PC & Peripherals', 'Make up', 'Pakistani Clothing', "Men's Accessories", 'Kitchen & Dining', "Girls' Fashion", 'Western Clothing ', 'Fragrances', 'Mobile & Tablet Accessories', "Men's Watches", 'Toys & Games', 'Cooking Essentials', 'Health Care ', 'Home Decor', 'Accessories', 'Watches', 'Bath', 'Team Sports', 'Racket Sports', 'Winter Wear']
49


In [7]:
indexes=[]
length=len(keep_labels)
for i in range(length):
    index=np.argwhere(keep_labels[i]==ytrain)
    indexes.extend(index)
print len(indexes)
Xtrain=Xtrain[indexes]
ytrain=ytrain[indexes]

149399


In [8]:
unique_classes=len(np.unique(ytrain))

In [9]:
print unique_classes

49


In [10]:
print type(keep_labels)
print ytrain.shape
ytrain=np.squeeze(ytrain)

<type 'list'>
(149399, 1)


In [11]:
from sklearn.utils.class_weight import compute_class_weight
print len(np.unique(ytrain))
print len(keep_labels)
print np.sort(np.unique(ytrain))
print np.sort(keep_labels)
weights=compute_class_weight('balanced',np.sort(keep_labels),np.sort(ytrain))
print weights

49
49
['Accessories' 'Audio & Video' "Babies' Fashion" 'Baby & Toddler' 'Bath'
 'Bath & Body' 'Bedding' 'Books & Stationery' "Boys' Fashion" 'Cameras'
 'Components & Spare Parts' 'Cooking Essentials' 'Fitness Accessories'
 'Fragrances' 'Furniture' 'Gaming PC & Peripherals' "Girls' Fashion"
 'Hair Care' 'Health Care ' 'Home Decor' 'Home Improvements'
 'Kitchen & Dining' 'Large Appliances' 'Lighting' 'Lingerie & Sleepwear'
 'Make up' "Men's Accessories" "Men's Shoes" "Men's Watches"
 'Mobile & Tablet Accessories' 'Outdoor Activities' 'Pakistani Clothing'
 'Peripherals & Accessories' 'Personal Care' 'Racket Sports' 'Shoes'
 'Shoes & Clothing' 'Skin Care' 'Small Appliances' 'Storage'
 'Strength Training Equipment' 'Team Sports' 'Toys & Games' 'Video Games'
 'Watches' 'Western Clothing' 'Western Clothing ' 'Winter Clothing'
 'Winter Wear']
['Accessories' 'Audio & Video' "Babies' Fashion" 'Baby & Toddler' 'Bath'
 'Bath & Body' 'Bedding' 'Books & Stationery' "Boys' Fashion" 'Cameras'
 'Compon

In [12]:
importance={}
keep_labels=np.sort(keep_labels)
for idx,i in enumerate(keep_labels):
    importance[i]=weights[idx]
print importance

{'Books & Stationery': 1.0668156695848388, 'Strength Training Equipment': 1.1886780443171421, "Men's Shoes": 0.84858312932743374, 'Bedding': 2.0926281288081463, 'Cameras': 2.5926523670692765, 'Storage': 2.6015010099602982, 'Accessories': 0.7362857241423495, 'Bath & Body': 2.1793846916893993, 'Hair Care': 0.71087880244193735, 'Peripherals & Accessories': 0.90205893008090809, 'Large Appliances': 1.5871729222662516, "Men's Accessories": 0.63164681658866162, 'Baby & Toddler': 0.96212028516045101, 'Video Games': 1.0769901743813033, 'Shoes & Clothing': 2.0019429965026063, "Boys' Fashion": 2.0138435823470737, 'Western Clothing': 0.54797972388092553, 'Audio & Video': 1.0535449839922146, 'Outdoor Activities': 1.3941285704954136, 'Personal Care': 0.95309758789417609, "Babies' Fashion": 1.2725205274096283, 'Lighting': 2.2287713330946413, 'Home Improvements': 2.0435383268588936, 'Furniture': 1.4945878351340536, 'Fitness Accessories': 1.7603690436913795, 'Winter Clothing': 1.1419322785293893, 'Skin

In [13]:
print importance['Accessories']

0.736285724142


In [14]:
labels=np.unique(ytrain)
le=LabelEncoder()
indexes=le.fit_transform(labels)
mydict_labels={}
temp={}
for idx,i in enumerate(labels):
    mydict_labels[i]=indexes[idx]
    temp[indexes[idx]]=importance[i]
print mydict_labels
print temp

{'Books & Stationery': 7, 'Strength Training Equipment': 40, "Men's Shoes": 27, 'Bedding': 6, 'Cameras': 9, 'Storage': 39, 'Accessories': 0, 'Bath & Body': 5, 'Hair Care': 17, 'Peripherals & Accessories': 32, 'Large Appliances': 22, "Men's Accessories": 26, 'Baby & Toddler': 3, 'Video Games': 43, 'Shoes & Clothing': 36, "Boys' Fashion": 8, 'Western Clothing': 45, 'Audio & Video': 1, 'Outdoor Activities': 30, 'Personal Care': 33, "Babies' Fashion": 2, 'Lighting': 23, 'Home Improvements': 20, 'Furniture': 14, 'Fitness Accessories': 12, 'Winter Clothing': 47, 'Skin Care': 37, 'Gaming PC & Peripherals': 15, 'Pakistani Clothing': 31, 'Kitchen & Dining': 21, "Girls' Fashion": 16, 'Western Clothing ': 46, 'Shoes': 35, 'Fragrances': 13, 'Mobile & Tablet Accessories': 29, "Men's Watches": 28, 'Health Care ': 18, 'Toys & Games': 42, 'Cooking Essentials': 11, 'Small Appliances': 38, 'Home Decor': 19, 'Components & Spare Parts': 10, 'Watches': 44, 'Bath': 4, 'Lingerie & Sleepwear': 24, 'Make up': 

In [15]:
#importance=temp
#temp1={}
#keys=importance.keys()
#for i in keys:
#    zeros=np.zeros(len(keys))
#    zeros[i]=1
#    temp1[zeros]=importance[i]
#print temp1
importance=temp.copy()
print importance

{0: 0.7362857241423495, 1: 1.0535449839922146, 2: 1.2725205274096283, 3: 0.96212028516045101, 4: 1.5283003426934683, 5: 2.1793846916893993, 6: 2.0926281288081463, 7: 1.0668156695848388, 8: 2.0138435823470737, 9: 2.5926523670692765, 10: 2.4707935037872524, 11: 2.8283480368028475, 12: 1.7603690436913795, 13: 1.4429527608487787, 14: 1.4945878351340536, 15: 1.7196611301034797, 16: 1.5093857344918165, 17: 0.71087880244193735, 18: 2.8818139732263415, 19: 0.73557519509613256, 20: 2.0435383268588936, 21: 0.73046458640955181, 22: 1.5871729222662516, 23: 2.2287713330946413, 24: 1.1213531385338247, 25: 0.5928367069168714, 26: 0.63164681658866162, 27: 0.84858312932743374, 28: 1.2190960350553657, 29: 0.33802208244716958, 30: 1.3941285704954136, 31: 0.43463423858495642, 32: 0.90205893008090809, 33: 0.95309758789417609, 34: 1.3119445712880677, 35: 1.4693779198426358, 36: 2.0019429965026063, 37: 1.4402263503417427, 38: 0.5582129592957652, 39: 2.6015010099602982, 40: 1.1886780443171421, 41: 1.419440960

In [16]:
ytrain_=[]
for i in range(ytrain.shape[0]):
    ytrain_.append(mydict_labels[ytrain[i]])
ytrain=np.array(ytrain_)
print ytrain
print ytrain.shape

[ 7  7  7 ..., 48 48 48]
(149399,)


In [17]:
unique_labels=mydict_labels.values()
length_unique_labels=len(unique_labels)
temp1=[]
temp2=[]
length=0
for i in range(length_unique_labels):
    indexes=np.argwhere(unique_labels[i]==ytrain)
    length=len(indexes)
    index=np.arange(length)
    np.random.shuffle(index)
    indexes=indexes[index]
    temp1.extend(indexes[:int(0.1*length)])
    temp2.extend(indexes[int(0.1*length):])
print length
print len(temp1)
print len(temp2)


2981
14918
134481


In [18]:
Xtest=Xtrain[temp1]
ytest=ytrain[temp1]
Xtrain=Xtrain[temp2]
ytrain=ytrain[temp2]
Xtrain=np.squeeze(Xtrain)
Xtest=np.squeeze(Xtest)
ytest=np.squeeze(ytest)
ytrain=np.squeeze(ytrain)
print Xtest.shape
print Xtrain.shape
print ytrain.shape
print ytest.shape

(14918, 229, 229, 3)
(134481, 229, 229, 3)
(134481,)
(14918,)


In [19]:
sample=[]
for i in ytest:
    imp=importance[i]
    sample.append(imp)

In [20]:
labels=[]
for i in (ytrain):
    zeros=np.zeros(unique_classes,dtype=np.int8)
    zeros[i]=1
    labels.append(zeros)
ytrain=np.array(labels)

print ytrain.shape
labels=[]
for i in (ytest):
    zeros=np.zeros(unique_classes,dtype=np.int8)
    zeros[i]=1
    labels.append(zeros)
ytest=np.array(labels)
print ytest.shape

(134481, 49)
(14918, 49)


In [37]:
print ytest

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 1]
 [0 0 0 ..., 0 0 1]
 [0 0 0 ..., 0 0 1]]


labels=np.array(labels,dtype=np.int8)
print labels.shape

from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest=train_test_split(Xtrain,labels)

print Xtrain.shape
print Xtest.shape
print ytrain.shape
print ytest.shape

Xtrain=np.squeeze(Xtrain)
Xtest=np.squeeze(Xtest)

print Xtrain.shape
print Xtest.shape
print ytrain.shape
print ytest.shape

In [ ]:
np.savez_compressed("val1",a=Xtest,b=ytest)

In [ ]:
np.savez_compressed("train1",a=Xtrain,b=ytrain)

In [21]:
from keras.applications.inception_v3 import InceptionV3
base_model = InceptionV3(weights='imagenet', include_top=False)

In [22]:
print np.max(Xtrain)
print np.min(Xtrain) 
print np.max(Xtest)
print np.min(Xtest)

127
-128
127
-128


N,H,W,C=Xtrain.shape
for i in range(N):
    for j in range(H):
        for k in range(W):
            for l in range(C):
                Xtrain[i,j,k,l]+=128
                #print Xtrain[i,j,k,l]
                #break
            #break
        #break
        
    #break
N,H,W,C=Xtest.shape
for i in range(N):
    for j in range(H):
        for k in range(W):
            for l in range(C):
                Xtest[i,j,k,l]+=128

In [23]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [24]:
def add_new_last_layer(base_model, nb_classes):
    """Add last layer to the convnet
    Args:
    base_model: keras model excluding top layer
    nb_classes: number of classes
    Returns:
    new keras model with last fully connected layer
    """
    FC_SIZE=1024
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(FC_SIZE, activation='relu')(x) 
    predictions = Dense(nb_classes, activation='softmax')(x) 
    model = Model(input=base_model.input, output=predictions)
    return model

In [25]:
def setup_to_transfer_learn(model, base_model):
    """Freeze all layers and compile the model"""
    for layer in base_model.layers:
        layer.trainable = False
    adam=Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

In [26]:
def setup_to_finetune(model):
    """Freeze the bottom NB_IV3_LAYERS and retrain the remaining top 
      layers.
    note: NB_IV3_LAYERS corresponds to the top 2 inception blocks in 
         the inceptionv3 architecture
    Args:
     model: keras model
    """
    for layer in model.layers[:NB_IV3_LAYERS_TO_FREEZE]:
        layer.trainable = False
    for layer in model.layers[NB_IV3_LAYERS_TO_FREEZE:]:
        layer.trainable = True
    adam=Adam(lr=3e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08,decay=0.1)
    model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

In [27]:
model=add_new_last_layer(base_model,49)
#setup_to_transfer_learn(model,base_model)
setup_to_finetune(model)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

/home/i140069/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  


In [28]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [29]:
model.load_weights("weights-improvement-05-0.9795.hdf5")

In [30]:
res=model.fit(Xtrain,ytrain,batch_size=128,epochs=5,validation_data=(Xtest,ytest),callbacks=callbacks_list,
              class_weight=importance)

Train on 134481 samples, validate on 14918 samples
Epoch 1/5
134481/134481 [==============================] - 1835s 14ms/step - loss: 0.9794 - acc: 0.6961 - val_loss: 0.9176 - val_acc: 0.7235
Epoch 2/5
134481/134481 [==============================] - 1803s 13ms/step - loss: 0.9448 - acc: 0.7060 - val_loss: 0.9148 - val_acc: 0.7252
Epoch 3/5
134481/134481 [==============================] - 1802s 13ms/step - loss: 0.9408 - acc: 0.7074 - val_loss: 0.9136 - val_acc: 0.7257
Epoch 4/5
134481/134481 [==============================] - 1801s 13ms/step - loss: 0.9374 - acc: 0.7085 - val_loss: 0.9130 - val_acc: 0.7242
Epoch 5/5
134481/134481 [==============================] - 1800s 13ms/step - loss: 0.9338 - acc: 0.7093 - val_loss: 0.9122 - val_acc: 0.7256


In [31]:
predictions=model.predict(Xtest)
indexes=np.argsort(predictions)
print indexes.shape

(14918, 49)


In [32]:
temp=indexes[:,46:]
print temp.shape

(14918, 3)


In [33]:
print ytest

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 1]
 [0 0 0 ..., 0 0 1]
 [0 0 0 ..., 0 0 1]]


In [34]:
print temp

[[11 13  7]
 [33 29  7]
 [ 2  7 42]
 ..., 
 [31 46 48]
 [27 46 48]
 [27 46 48]]


In [38]:
counter=0
for i in range(ytest.shape[0]):
    if np.argwhere(ytest[i]==1) in temp[i]:
        counter+=1
print counter

13856


In [39]:
print "acuuracy = ",counter/float(ytest.shape[0])*100

acuuracy =  92.8810832551
